#### Fetching Latest Fuel Data from NSW Endpoint
Prepared by Edward Ho

The purpose of this notebook is to provide a framework to fetch latest data from NSW endpoints, and to pre-process some of the data for users requests and/or other analytical purposes.

In [1]:
import os
import datetime
import json

import requests
import pandas as pd

import plotly.express as px

In [7]:
class FuelPrice:
    def __init__(self, authorization, is_update=False):
        self.is_update = is_update
        self.authorization = authorization
        self.access_token = None
        self.raw_response = None
        self.stations_df = None
        self.prices_df = None
        self.combined_df = None

    def get_token(self):
        print('Step 1/6 : Generating Access Token for NSW API...')
        url = "https://api.onegov.nsw.gov.au/oauth/client_credential/accesstoken"

        querystring = {"grant_type":"client_credentials"}

        headers = {
            'content-type': "application/json",
            'authorization': self.authorization
            }

        response = requests.request("GET", url, headers=headers, params=querystring)

        self.access_token = response.json()['access_token']
        
    def update_data(self):
        print('Step 2/6 : Fetching Fuel Price from NSW API...')
        url = "https://api.onegov.nsw.gov.au/FuelPriceCheck/v2/fuel/prices"

        querystring = {"states":"NSW"}
        headers = {
            'content-type': CONTENT_TYPE,
            'authorization': f'Bearer {self.access_token}',
            'apikey': API_KEY,
            'transactionid': '1',
            'requesttimestamp': CUR_TIME,
            }
        try:
            response = requests.request("GET", url, headers=headers, params=querystring)
            self.raw_response = response.json()
        except:
            print('Failed to fetch fuel prices, please try again later.')
            
    # Populating Price List
    def create_price_list(self):
        print('Step 3/6 : Generating Fuel Price Dataframe...')
        self.prices_df = pd.DataFrame.from_dict(self.raw_response['prices'])

    # Create Table for Station Mapping
    def create_station_list(self):
        print('Step 4/6 : Generating Station List... ')
        rows = []
        for station in self.raw_response['stations']:
            row = []
            for col, value in station.items():
                if col == 'location':
                    for _, degree in value.items():
                        row.append(degree)
                else:
                    row.append(value)
            rows.append(row)
        df = pd.DataFrame(rows, columns = ['brandid', 'stationid', 'brand', 'code', 'name',
                                                    'address', 'latitude', 'longitude', 'state'])
        df['code'] = df['code'].astype('int64')
        self.stations_df = df.copy()
        
    def build_working_df(self):
        print('Step 5/6 : Building Working Dataframe for Processing... ')
        self.working_df = pd.merge(self.stations_df,
                                   self.prices_df,
                                   how='right',
                                   left_on='code',
                                   right_on='stationcode')
    def save_as_csv(self, path):
        self.stations_df.to_csv(f'{path}/stations.csv', index=True)
        self.prices_df.to_csv(f'{path}/prices.csv', index=True)
        self.working_df.to_csv(f'{path}/combined.csv', index=True)
        
        print('   - Export Completed.')
    
    def check_folder(self, path):
        if os.path.exists(path):
            self.save_as_csv(path)
        else:
            os.mkdir(path)
            self.save_as_csv(path)
    
    def export(self):
        print('Step 6/6 : Saving Price and Station data as csv file... ')
        today_date = datetime.datetime.now().strftime('%Y%m%d')
        path = f'data/backup_{today_date}'
        if os.path.exists('data'):
            self.check_folder(path)
        else:
            os.mkdir('data')
            self.check_folder(path)
            
    def read_record(self):
        all_subdirs = sorted([d for d in os.listdir('data')], reverse=True)
        latest_folder = all_subdirs[0]
        
        path = f'data/{latest_folder}'
        
        print(f'Step 1/3 : Retriving Fuel Price Data on {latest_folder[-8:]}')
        self.prices_df = pd.read_csv(f'{path}/prices.csv', index_col=False)
        print(f'Step 2/3 : Retriving Fuel Price Data on {latest_folder[-8:]}')
        self.stations_df = pd.read_csv(f'{path}/stations.csv', index_col=False)
        print(f'Step 3/3 : Retriving Fuel Price Data on {latest_folder[-8:]}')
        self.working_df = pd.read_csv(f'{path}/combined.csv', index_col=False)
        
        print('Data retrieved without error.')
            
    def run(self):
        print('FuelRun - Backend //')
        if self.is_update:
            print('Downloading and Processing Latest Data - ')
            self.get_token()
            self.update_data()
            self.create_price_list()
            self.create_station_list()
            self.build_working_df()
            self.export()
        else:
            self.read_record()
            

In [8]:
fuel = FuelPrice(BASE64_AUTH, is_update=True)
fuel.run()

FuelRun - Backend //
Step 1/6 : Generating Access Token for NSW API...
Step 2/6 : Fetching Fuel Price from NSW API...
Step 3/6 : Generating Fuel Price Dataframe...
Step 4/6 : Generating Station List... 
Step 5/6 : Building Working Dataframe for Processing... 
Step 6/6 : Saving Price and Station data as csv file... 
   - Export Completed.


In [31]:
E10_price_by_company = fuel.working_df[fuel.working_df['fueltype'] == 'E10'].groupby('brand').mean().reset_index()
P98_price_by_company = fuel.working_df[fuel.working_df['fueltype'] == 'P98'].groupby('brand').mean().reset_index()

In [43]:
E10_price_by_company.describe()

,code,latitude,longitude,stationcode,price
count,19.000000,19.000000,19.000000,19.000000,19.000000
mean,7408.331389,-33.334499,150.715260,7408.331389,205.911887
std,4074.012438,0.714319,0.618715,4074.012438,6.585020
min,947.292398,-34.106348,148.666023,947.292398,187.366667
25%,4285.299170,-33.727614,150.650246,4285.299170,202.752113
50%,7389.333333,-33.542539,150.908288,7389.333333,206.417647
75%,10928.740260,-33.400756,151.031275,10928.740260,209.954251
max,14108.416667,-31.417353,151.514022,14108.416667,215.110526


In [44]:
P98_price_by_company.describe()

,code,latitude,longitude,stationcode,price
count,22.000000,22.000000,22.000000,22.000000,22.000000
mean,6769.994967,-33.143551,150.390396,6769.994967,227.316448
std,4236.432248,0.960379,0.942110,4236.432248,7.770802
min,876.536458,-34.379865,147.895125,876.536458,203.700000
25%,4046.840054,-33.737619,150.080695,4046.840054,222.757007
50%,6517.404847,-33.497245,150.697532,6517.404847,228.832500
75%,9877.360151,-32.861066,151.008055,9877.360151,230.204179
max,17536.000000,-31.037935,151.366461,17536.000000,239.530208


In [39]:
E10_plot_price_by_company = price_by_company[~(E10_price_by_company['brand'].isin(['Tesla','NRMA']))]


In [40]:
P98_plot_price_by_company = price_by_company[~(P98_price_by_company['brand'].isin(['Tesla','NRMA']))]

/Users/edward/Documents/Programming/fuelrun/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [41]:
import plotly.express as px
fig = px.bar(E10_plot_price_by_company,
       x = 'brand', y='price',
       title='Average E10 Prices by Company',
       text_auto='.5s').update_xaxes(categoryorder="total descending")
fig.show()